This notebook completes the PanTaGruEl model as found at [https://github.com/laurentpagnier/PanTaGruEl.jl](https://github.com/laurentpagnier/PanTaGruEl.jl)

It relies on some data that can be downloaded from the [ENTSO-E transparency platform](https://transparency.entsoe.eu/) using SFTP. The data must be stored in the following directory:

In [1]:
import os
entsoe_data_dir = os.path.expanduser('~/data/entso-e/raw')

# Import PanTaGruEl model

Use the JSON file located in the parent directory:

In [2]:
import json

In [3]:
with open('../pantagruel.json') as f:
    model = json.load(f)

In [4]:
{key: len(values) for key, values in model.items() if isinstance(values, dict)}

{'bus': 4097,
 'dcline': 0,
 'gen': 1083,
 'branch': 8375,
 'storage': 0,
 'switch': 0,
 'shunt': 0,
 'load': 3998}

# Compute susceptance for all lines

The susceptance $B$ is given in terms of the resistance $R$ and reactance $X$ by the formula
$$B = \frac{X}{R^2 + X^2}$$

In [5]:
for line in model['branch'].values():
    line['br_b'] = line['br_x'] / (line['br_x']**2 + line['br_r']**2)

# Fix missing bus locations

Some buses of the PanTaGruEl model file are missing the country information. This can be fixed by hand based on the latitude and longitude.

In [6]:
bus_location_fix = {
    "7321": "FR",
    "7578": "FR",
    "7015": "FR",
    "7586": "FR",
    "7788": "FR",
    "1482": "IT",
    "7822": "FR",
    "7380": "FR",
    "7336": "FR",
    "7333": "FR",
    "7327": "FR",
    "7175": "PT",
    "6988": "DK",
    "7334": "FR",
    "6873": "DK",
    "7821": "FR",
    "7575": "FR",
    "7750": "FR",
    "7337": "FR",
    "7715": "FR",
    "7339": "FR",
    "7741": "FR",
    "6949": "FR",
    "6943": "FR",
    "7322": "FR",
    "7326": "FR"
}

In [7]:
for (id, country) in bus_location_fix.items():
    model["bus"][id]["country"] = country

# Remove small generators

In [8]:
# [(id, gen['type'], gen['pmax']) for id, gen in sorted(model['gen'].items(), key=lambda item: item[1]['pmax'])]

In [9]:
# import matplotlib.pyplot as plt
# plt.hist([gen['pmax'] for (id, gen) in model['gen'].items()], bins=100, range=(0, 2))
# plt.show()

All generators with output below 50 MW are removed:

In [10]:
low_power_gens = [id for (id, gen) in model['gen'].items() if gen['pmax'] < 0.5]
len(low_power_gens)

268

In [11]:
for id in low_power_gens:
    del model['gen'][id]

The buses' type must be updated in some cases:

In [12]:
buses_with_gens = {str(gen['gen_bus']) for gen in model['gen'].values()}

In [13]:
for id, bus in model['bus'].items():
    if bus['bus_type'] == 2 and id not in buses_with_gens:
        bus['bus_type'] = 1

# Add a load at every bus

Some buses do not have an associated load in PanTaGruEl.

In [14]:
all_buses = set(model['bus'].keys())
len(all_buses)

4097

In [15]:
buses_with_loads = {str(load['load_bus']) for load in model['load'].values()}
len(buses_with_loads)

3998

Map between bus and load:

In [16]:
load_by_bus = {str(load['load_bus']): load['pd'] for load in model['load'].values()}

Re-assign loads with IDs matching the buses, using the given value for existing loads, and otherwise zero.

In [17]:
model['load'] = {id: {
                    'index'   : int(id),
                    'load_bus': int(id),
                    'status'  : 1,
                    'pd'      : load_by_bus[id] if id in load_by_bus else 0.0,
                    'qd'      : 0
                } for id in all_buses}

Ensure that the 'load_prop' properties add up to one for every country:

In [18]:
load_prop_by_country = {bus['country']: 0.0 for bus in model['bus'].values()}
for bus in model['bus'].values():
    load_prop_by_country[bus['country']] += bus['load_prop']

{country: value for country, value in load_prop_by_country.items() if abs(value - 1.0) > 1e-6}

{'XX': 0.0}

In [19]:
n_bus_XX = [bus["country"] for bus in model["bus"].values()].count('XX')
n_bus_XX

13

In [20]:
for bus in model['bus'].values():
    if bus['country'] == 'XX':
        bus['load_prop'] = 1 / n_bus_XX

# Add ENTSO-E generator names

Add temporary unique generator name for all gens of the model, as well as the generators country:

In [21]:
for gen_id, gen in model['gen'].items():
    bus_id = str(gen['gen_bus'])
    gen['name'] = model['bus'][bus_id]['name'] + ' ' + gen_id
    gen['country'] = model['bus'][bus_id]['country']

In [22]:
all_gen_names = {gen['name'] for gen in model['gen'].values()}

Import list of generators from ENTSO-E at a given reference time (January 2016)

In [23]:
import pandas as pd

In [24]:
entsoe_data = pd.DataFrame(pd.read_csv('%s/2016_01_ActualGenerationOutputPerGenerationUnit_16.1.A.csv' % entsoe_data_dir, sep='\t') \
    .groupby(['PowerSystemResourceName', 'MapCode', 'ProductionType', 'InstalledGenCapacity'])['ActualGenerationOutput'].mean()) \
    .reset_index()
entsoe_data

,PowerSystemResourceName,MapCode,ProductionType,InstalledGenCapacity,ActualGenerationOutput
0,ABOÑO 1,ES,Fossil Hard coal,341.7,155.520027
1,ABOÑO 2,ES,Fossil Hard coal,535.8,411.376075
2,ABTH7,GB,Fossil Hard coal,535.0,501.421135
3,ABTH8,GB,Fossil Hard coal,535.0,483.604071
4,ABTH9,GB,Fossil Hard coal,535.0,395.362667
...,...,...,...,...,...
1679,Zydowo H3,PL,Hydro Pumped Storage,54.0,2.414234
1680,iPower Solutions ltd,NIE,Fossil Oil,74.0,0.034341
1681,Åbyverket Örebro,SE,Biomass,106.0,29.132615
1682,Öresundsverket CHP G1,SE,Fossil Gas,294.0,210.140970


Determine country from MapCode column:

In [25]:
entsoe_country_map = {mapcode: mapcode for mapcode in entsoe_data['MapCode']}
for mapcode in entsoe_country_map.keys():
    if mapcode[0:2] == 'DE':
        entsoe_country_map[mapcode] = 'DE'
entsoe_data.insert(2, 'Country', list(map(lambda mapcode: entsoe_country_map[mapcode], entsoe_data['MapCode'])))

Filter generators that are in one of the model's country (ENTSO-E data also containes UK, Ireland, Sweden...)

In [26]:
model_countries = {gen['country'] for gen in model['gen'].values()}
model_countries

{'AL',
 'AT',
 'BA',
 'BE',
 'BG',
 'CH',
 'CZ',
 'DE',
 'DK',
 'ES',
 'FR',
 'GR',
 'HR',
 'HU',
 'IT',
 'LU',
 'ME',
 'MK',
 'NL',
 'PL',
 'PT',
 'RO',
 'RS',
 'SI',
 'SK',
 'XX'}

In [27]:
entsoe_data = entsoe_data[entsoe_data['Country'].isin(model_countries)]

### Switzerland

Manually examine ENTSO-E generators list:

In [28]:
CH_name_map = {
    'Beznau 943': ['Beznau 1', 'Beznau 2'],
    'Mühleberg 917': ['KKM Produktion'],
    'Leibstadt 908': ['Leibstadt'],
    'Gösgen 921': ['Kernkraftwerk Gösgen'],
    'Biasca 938': ['Centrale di Biasca'],
    'Veytaux 913': ['Usine de Veytaux'],
    'Bâtiaz 912': ['Usine de la Bâtiaz'],
    'Chamoson 914': ['Usine de Bieudron', 'Usine de Nendaz'],
    'Fionnay GD 911': ['Usine de Fionnay'],
    'Verbano 922': ['Centrale di Verbano'],
    'Robiei 936': ['Centrale di Robiei'],
    'Bitsch 916': ['Kraftwerk Bitsch'],
    'Bavona 935': ['Centrale di Bavona'],
    'Cavergno 923': ['Centrale di Cavergno'],
    'Mapragg 939': ['Mapragg - Gigerwald G1'],
    'Grimsel 909': ['KWO Produktion'],
    'Innertkirchen 918': ['KWO Produktion'],
    'Riddes 173': ['Stufe FMM Produktion Total'],
    'Fionnay FMM 910': ['Stufe FMM Produktion Total'],
    'Limmern 958': ['Limmern - Muttsee G1'],
    'Tierfehd 926': ['Tierfehd - Limmern GPSW'],
    'Olivone 925': ['AET Leventina']
}

In [29]:
CH_name_values = {name for list_of_names in CH_name_map.values() for name in list_of_names}

Check that the keys are part of the model's names:

In [30]:
assert set(CH_name_map.keys()).issubset(all_gen_names)

Check that the names belong to the ENTSO-E names:

In [31]:
assert CH_name_values.issubset(entsoe_data['PowerSystemResourceName'])

Model's generators not in the list:

In [32]:
pd.DataFrame([[gen['name'], gen['type'], 100 * gen['pmax']] for gen in model['gen'].values()
             if gen['country'] == 'CH' and gen['name'] not in CH_name_map.keys()],
             columns=['Name', 'Type', 'Pmax']).sort_values('Name')

,Name,Type,Pmax
11,Bärenburg 930,hydro_pure_storage,220.0
0,Castasegna 928,hydro_pure_storage,100.0
3,Ferrera 929,hydro_mixed,180.0
9,Foretaille 766,hydro_ror,98.0
5,Göschenen 919,hydro_pure_storage,160.0
6,Handeck 937,hydro_mixed,113.0
2,Laufenburg 920,hydro_ror,106.0
12,Löbbia 933,hydro_pure_storage,95.0
1,Pradella 934,hydro_pure_storage,288.0
4,Rothenbrunnen 932,hydro_pure_storage,127.0


ENTSO-E generators not in the list:

In [33]:
entsoe_data[(entsoe_data['Country'] == 'CH') & (~entsoe_data['PowerSystemResourceName'].isin(CH_name_values))] \
    [['PowerSystemResourceName', 'ProductionType', 'InstalledGenCapacity']]

,PowerSystemResourceName,ProductionType,InstalledGenCapacity
776,KW Rheinfelden CH,Hydro Run-of-river and poundage,100.0
1580,Usine de Vallorcine,Hydro Water Reservoir,230.0


### France

Manually examine ENTSO-E generators list:

In [34]:
FR_name_map = {
    'Belleville 757': ['BELLEVILLE 1', 'BELLEVILLE 2'],
    'Boctois 715': ['NOGENT 1', 'NOGENT 2'],
    'Cattenom 732': ['CATTENOM 1', 'CATTENOM 2', 'CATTENOM 3', 'CATTENOM 4'],
    'Chinon B 751': ['CHINON 1', 'CHINON 2', 'CHINON 3', 'CHINON 4'],
    'Chooz 518': ['CHOOZ 1', 'CHOOZ 2'],
    'Civaux 602': ['CIVAUX 1', 'CIVAUX 2'],
    'Coulange 657': ['CRUAS 1', 'CRUAS 2', 'CRUAS 3', 'CRUAS 4'],
    'Creys 690': ['BUGEY 2', 'BUGEY 3', 'BUGEY 4', 'BUGEY 5'],
    'Dampierre 708': ['DAMPIERRE 1', 'DAMPIERRE 2', 'DAMPIERRE 3', 'DAMPIERRE 4'],
    'Fessenheim 318': ['FESSENHEIM 1', 'FESSENHEIM 2'],
    'Flamanville 982': ['FLAMANVILLE 1', 'FLAMANVILLE 2'],
    'Golfech 552': ['GOLFECH 1', 'GOLFECH 2'],
    'Gravelines 723': ['GRAVELINES 1', 'GRAVELINES 2', 'GRAVELINES 3', 'GRAVELINES 4', 'GRAVELINES 5', 'GRAVELINES 6'],
    'Le Blayais 583': ['BLAYAIS 1', 'BLAYAIS 2', 'BLAYAIS 3', 'BLAYAIS 4'],
    'Paluel 718': ['PALUEL 1', 'PALUEL 2', 'PALUEL 3', 'PALUEL 4'],
    'Penly Poste 717': ['PENLY 1', 'PENLY 2'],
    'P.Cordier 670': ['ST ALBAN 1', 'ST ALBAN 2'],
    'St. Laurent 554': ['ST LAURENT 1', 'ST LAURENT 2'],
    'Tricastin 658': ['TRICASTIN 1', 'TRICASTIN 2', 'TRICASTIN 3', 'TRICASTIN 4'],
    'Cordemais Poste 986': ['CORDEMAIS 2', 'CORDEMAIS 3', 'CORDEMAIS 4', 'CORDEMAIS 5'],
    'Porcheville 711': ['PORCHEVILLE 1', 'PORCHEVILLE 2', 'PORCHEVILLE 3', 'PORCHEVILLE 4'],
    'Aramon 556': ['ARAMON 1', 'ARAMON 2'],
    'Villarodin 695': ['VILLARODIN 1', 'VILLARODIN 2'],
    'Le Pouget 553': ['POUGET 4'],
    'Brommat 758': ['BROMMAT 7'],
    'Le Cheylas 685': ['CHEYLAS 1', 'CHEYLAS 2'],
    'S. Bissorte 765': ['SUPER BISSORTE 1', 'SUPER BISSORTE 2', 'SUPER BISSORTE 3', 'SUPER BISSORTE 4', 'SUPER BISSORTE 5'],
    'Montezic poste et centrale 759': ['MONTEZIC 1', 'MONTEZIC 2', 'MONTEZIC 3', 'MONTEZIC 4'],
    'Revin 726': ['REVIN 1', 'REVIN 2', 'REVIN 3', 'REVIN 4'],
    'Guersac 987': ['SPEM CCG'],
    'Quartes 725': ['Pont-sur-Sambre'],
    'Braek 555': ['DK6-TG1', 'DK6-TG2', 'DK6-TV1', 'DK6-TV2'],
    'Ratier 754': ['AMFARD14', 'AMFARD15'],
    'Darse 610': ['COMBIGOLFE CCG', 'CYCOFOS PL1', 'CYCOFOS PL2'],
    'Ponteau 955': ['MARTIGUES PONTEAU 5', 'MARTIGUES PONTEAU 6'],
    'Custines 749': ['Croix-de-Metz'],
    'St. Avold 731': ['EMILE HUCHET 6', 'EMILE HUCHET 7', 'EMILE HUCHET 8'],
    'Bezaumont 729': ['BLENOD 5'],
    'Le Havre 603': ['HAVRE 4'],
    'La Palun 672': ['PROVENCE 5'],
    'Arrighi 709': ['ARRIGHI 1', 'ARRIGHI 2'],
    'Brennilis 985': ['BRENNILIS 1', 'BRENNILIS 2', 'BRENNILIS 3'],
    'Loscoat 983': ['DIRINON 1', 'DIRINON 2'],
    'Le Chesnoy 714': ['MONTEREAU 5', 'MONTEREAU 6'],
    'Villevaude 716': ['VAIRES 1', 'VAIRES 2', 'VAIRES 3'],
    'Sisteron 677': ['SISTERON 1', 'SISTERON 2'],
    'Le Chastang 612': ['CHASTANG 2', 'CHASTANG 3']
}

In [35]:
FR_name_values = {name for list_of_names in FR_name_map.values() for name in list_of_names}

Check that the keys are part of the model's names:

In [36]:
assert set(FR_name_map.keys()).issubset(all_gen_names)

Check that the names belong to the ENTSO-E names:

In [37]:
assert FR_name_values.issubset(entsoe_data['PowerSystemResourceName'])

Model's generators not in the list:

In [38]:
pd.DataFrame([[gen['name'], gen['type'], 100 * gen['pmax']] for gen in model['gen'].values()
             if gen['country'] == 'FR' and gen['name'] not in FR_name_map.keys()],
             columns=['Name', 'Type', 'Pmax']).sort_values('Name')

,Name,Type,Pmax
51,Aston 557,hydro_ror,104.0
17,Aussois 696,hydro_ror,87.0
37,Beaucaire 654,hydro_ror,210.0
50,Beauchastel 661,hydro_ror,222.0
35,Bissy 691,hydro_ror,91.0
...,...,...,...
48,St. Tulle 674,hydro_ror,88.0
15,Strasbourg 739,hydro_ror,147.0
45,Vallorcine 741,hydro_ror,190.0
16,Vogelgrun 737,hydro_ror,141.0


ENTSO-E generators not in the list:

In [39]:
entsoe_data[(entsoe_data['Country'] == 'FR') & (~entsoe_data['PowerSystemResourceName'].isin(FR_name_values))] \
    [['PowerSystemResourceName', 'ProductionType', 'InstalledGenCapacity']]

,PowerSystemResourceName,ProductionType,InstalledGenCapacity
15,AIGLE 6,Hydro Water Reservoir,146.0
118,BORT 1,Hydro Water Reservoir,118.0
119,BORT 2,Hydro Water Reservoir,114.0
245,COMBE D'AVRIEUX 1,Hydro Water Reservoir,123.0
548,FR-GA-MORANT1,Fossil Gas,401.0
569,GENNEVILLIERS 1,Fossil Gas,203.0
585,GRAND MAISON 1,Hydro Pumped Storage,152.0
586,GRAND MAISON 10,Hydro Pumped Storage,157.0
587,GRAND MAISON 11,Hydro Pumped Storage,157.0
588,GRAND MAISON 12,Hydro Pumped Storage,157.0


### Spain

Manually examine ENTSO-E generators list:

In [40]:
ES_name_map = {
    'Asco 79': ['ASCO 1', 'ASCO 2'],
    'C.N. Almaraz 160': ['ALMARAZ 1', 'ALMARAZ 2'],
    'Sagunto 87': ['COFRENTES'],
    'La Robla 1005': ['ROBLA 1', 'ROBLA 2'],
    'Soto de Ribera 1013': ['SOTO RIB 2', 'SOTO RIB 3'],
    'P.G. Rodriguez 998': ['P.G.RODR 1', 'P.G.RODR 2', 'P.G.RODR 3', 'P.G.RODR 4'],
    'Teruel 77': ['TERUEL 1', 'TERUEL 2', 'TERUEL 3'],
    'Lada 1009': ['LADA IV'],
    'Compostilla I 1004': ['ANLLARES 1', 'COMPOSTI 4', 'COMPOSTI 5'],
    'Belesar 997': ['BELESAR 1', 'BELESAR 2', 'BELESAR 3'],
    'G. y Galan 152': ['GGALAN GEN'],
}

In [41]:
ES_name_values = {name for list_of_names in ES_name_map.values() for name in list_of_names}

Check that the keys are part of the model's names:

In [42]:
assert set(ES_name_map.keys()).issubset(all_gen_names)

Check that the names belong to the ENTSO-E names:

In [43]:
assert ES_name_values.issubset(entsoe_data['PowerSystemResourceName'])

Model's generators not in the list:

In [44]:
pd.DataFrame([[gen['name'], gen['type'], 100 * gen['pmax']] for gen in model['gen'].values()
             if gen['country'] == 'ES' and gen['name'] not in ES_name_map.keys()],
             columns=['Name', 'Type', 'Pmax']).sort_values('Name')

,Name,Type,Pmax
32,Aguayo 1010,Hydro,360.60
24,Aldeadávila II 166,hydro_ror,820.00
34,Alvarado 158,other_nl,150.00
42,Aravaca 161,Hydro,61.95
22,Asco 78,Hydro,52.52
38,Azutan 149,hydro_ror,198.00
7,Cortes II 73,Hydro,70.17
23,Cortes II 74,Hydro,953.32
41,Cortes II 75,Hydro,634.00
28,Cortes II 76,Hydro,877.95


ENTSO-E generators not in the list:

In [45]:
entsoe_data[(entsoe_data['Country'] == 'ES') & (~entsoe_data['PowerSystemResourceName'].isin(ES_name_values))] \
    [['PowerSystemResourceName', 'ProductionType', 'InstalledGenCapacity']]

,PowerSystemResourceName,ProductionType,InstalledGenCapacity
0,ABOÑO 1,Fossil Hard coal,341.7
1,ABOÑO 2,Fossil Hard coal,535.8
5,ACECA 3,Fossil Gas,386.0
6,ACECA4,Fossil Gas,372.6
16,ALDEA I G1,Hydro Water Reservoir,132.9
...,...,...,...
1601,VLLRINO 4G,Hydro Water Reservoir,138.7
1602,VLLRINO 5B,Hydro Pumped Storage,138.0
1603,VLLRINO 5G,Hydro Water Reservoir,148.9
1604,VLLRINO 6B,Hydro Pumped Storage,138.0


### Germany

Manually examine ENTSO-E generators list:

In [46]:
DE_name_map = {
    'Brokdorf 210': ['Brokdorf'],
    'Gundremmingen 256': ['Gundremmingen B'],
    'Gundremmingen 257': ['Gundremmingen C'],
    'Isar 265': ['Isar 2'],
    'Philippsburg 232': ['KKW Philippsburg 2'],
    'Grohnde 209': ['Grohnde'],
    'Emsland 249': ['Emsland A'],
    'GKN 2 950': ['KKW Neckarwestheim 2'],
    'Boxberg 314': ['KW Boxberg Block N', 'KW Boxberg Block P', 'KW Boxberg Block Q', 'KW Boxberg Block R'],
    'Jänschwalde 312': ['KW Jänschwalde Block A', 'KW Jänschwalde Block B', 'KW Jänschwalde Block C',
                        'KW Jänschwalde Block D', 'KW Jänschwalde Block E', 'KW Jänschwalde Block F'],
    'Lippendorf 214': ['KW Lippendorf Block R', 'KW Lippendorf Block S'],
    'Neurath 238': ['Neurath A', 'Neurath B', 'Neurath C', 'Neurath D', 'Neurath E', 'Neurath F', 'Neurath G'],
    'Niederaußem 237': ['Niederaußem C', 'Niederaußem D', 'Niederaußem E', 'Niederaußem F',
                        'Niederaußem G', 'Niederaußem H', 'Niederaußem K (BoA 1)',
                        'Frimmersdorf P', 'Frimmersdorf Q',
                        'Weisweiler E', 'Weisweiler F', 'Weisweiler G', 'Weisweiler H'],
    'Schkopau 215': ['Schkopau A', 'Schkopau B'],
    'Buschhaus 213': ['Buschhaus'],
    'Schwarze Pumpe 313': ['KW Schwarze Pumpe Block A', 'KW Schwarze Pumpe Block B'],
    'Heilbronn 254': ['HKW Heilbronn Block 7'],
    'Rostock 304': ['Kraftwerk Rostock'], # incomplete,
    'Moorburg 319': ['HKW Moorburg Block A', 'HKW Moorburg Block B'],
    'Wilhelmshaven 204': ['Wilhelmshaven'],
    'Großkrotzenburg 272': ['Staudinger 5'], # incomplete
    'Bexbach 225': ['BEXBACH_A_GESAMT'],
    'Lippborg 248': ['BERGKAMEN_A'],
    'Ensdorf 948': ['Ensdorf 1', 'Ensdorf 3'],
    'Gersteinwerk 240': ['Gersteinwerk K2'],
    'Altbach 307': ['HKW Altbach/Deizisau Block 1', 'HKW Altbach/Deizisau Block 2'],
    'Reuter West 216': ['HKW Reuter West Block D', 'HKW Reuter West Block E'],
    'Kummerfeld 290': ['HKW Wedel Block 1', 'HKW Wedel Block 2'],
    'Ibbenbüren 309': ['Ibbenbüren B'],
    'Voerde 203': ['Kraftwerk Voerde Block A', 'Kraftwerk Voerde Block B'],
    'Scholven 243': ['Scholven B', 'Scholven C'], # incomplete
    'Ingolstadt 263': ['Ingolstadt 3', 'Ingolstadt 4'],
    'Vierraden 302': ['IKS Schwedt SE1 Block 1', 'IKS Schwedt SE2 Block 2']
}

In [47]:
DE_name_values = {name for list_of_names in DE_name_map.values() for name in list_of_names}

Check that the keys are part of the model's names:

In [48]:
assert set(DE_name_map.keys()).issubset(all_gen_names)

Check that the names belong to the ENTSO-E names:

In [49]:
assert DE_name_values.issubset(entsoe_data['PowerSystemResourceName'])

Model's generators not in the list:

In [50]:
pd.DataFrame([[gen['name'], gen['type'], 100 * gen['pmax']] for gen in model['gen'].values()
             if gen['country'] == 'DE' and gen['name'] not in DE_name_map.keys()],
             columns=['Name', 'Type', 'Pmax']).sort_values('Name')

,Name,Type,Pmax
39,Altenkleusheim 245,hydro_pure_ps,140.0
17,Altentreptow/Süd 972,Gas,75.0
16,BASF 270,fossil_coal_gas,460.0
67,BASF 271,fossil_coal_hard,870.0
64,Bergrheinfeld 281,hydro_pure_ps,160.0
...,...,...,...
6,Waldeck 273,hydro_pure_ps,578.0
5,Wehr 219,hydro_pure_ps,980.0
34,Weiher 202,fossil_brown_lignite,680.0
51,Wolmirstedt 292,Gas,52.0


ENTSO-E generators not in the list:

In [51]:
entsoe_data[(entsoe_data['Country'] == 'DE') & (~entsoe_data['PowerSystemResourceName'].isin(DE_name_values))] \
    [['PowerSystemResourceName', 'ProductionType', 'InstalledGenCapacity']]

,PowerSystemResourceName,ProductionType,InstalledGenCapacity
169,Block 1,Fossil Hard coal,136.0
170,Block 2,Fossil Hard coal,136.0
171,Block 3,Fossil Hard coal,690.0
172,Block AGuD,Fossil Gas,100.0
173,Block B,Fossil Gas,100.0
...,...,...,...
1659,Weisweiler VGT - BI. G,Fossil Gas,200.0
1660,Weisweiler VGT - BI. H,Fossil Gas,200.0
1662,Westfalen C,Fossil Hard coal,250.0
1663,Westfalen E,Fossil Hard coal,763.7


### Nuclear plants in other countries

Manually examine ENTSO-E generators list:

In [52]:
jaslovske = [name for name in all_gen_names if name[-3:] == '377'][0]

name_map = CH_name_map | FR_name_map | ES_name_map | DE_name_map | {
    jaslovske: ['Bohunice TG31', 'Bohunice TG32', 'Bohunice TG41', 'Bohunice TG42'],
    'Borssele 846': ['Borssele 30'],
    'Tihange 521': ['TIHANGE 1N', 'TIHANGE 1S', 'TIHANGE 2', 'TIHANGE 3'],
    'Mochovce 378': ['Mochovce TG11', 'Mochovce TG12', 'Mochovce TG21', 'Mochovce TG22'],
    'Cernavoda 404': ['CNE_U1_RET_CA', 'CNE_U2_RET_CA'],
    'Kallo 515': ['DOEL 1', 'DOEL 2', 'DOEL 3', 'DOEL 4'],
    'Dukovany 176': ['EDUK_B1____', 'EDUK_B2____', 'EDUK_B3____', 'EDUK_B4____'],
    'Temelín 179': ['ETEM_G1____', 'ETEM_G2____'],
    'Kozloduy 418': ['NPP KZL G9', 'NPP KZL G10'],
    'Paks 782': ['PA_gép1', 'PA_gép2', 'PA_gép3', 'PA_gép4', 'PA_gép5', 'PA_gép6', 'PA_gép7', 'PA_gép8'],
}

In [53]:
name_values = {name for list_of_names in name_map.values() for name in list_of_names}

Total number of model generators, total number of ENTSO-E generators:

In [54]:
len(name_map), len(name_values)

(123, 275)

Check that the keys are part of the model's names:

In [55]:
assert set(name_map.keys()).issubset(all_gen_names)

Check that the names belong to the ENTSO-E names:

In [56]:
assert name_values.issubset(entsoe_data['PowerSystemResourceName'])

Model's nuclear generators not in the list:

In [57]:
pd.DataFrame([[gen['name'], gen['type'], 100 * gen['pmax']] for gen in model['gen'].values()
             if 'nuclear' in gen['type'] and gen['name'] not in name_map.keys()],
             columns=['Name', 'Type', 'Pmax']).sort_values('Name')

,Name,Type,Pmax
1,Grafenrheinfeld 282,nuclear_cons,1275.0
0,Krško 886,nuclear,696.0


ENTSO-E nuclear generators not in the list:

In [58]:
entsoe_data[(entsoe_data['ProductionType'] == 'Nuclear') & (~entsoe_data['PowerSystemResourceName'].isin(name_values))] \
    [['PowerSystemResourceName', 'ProductionType', 'InstalledGenCapacity']]

,PowerSystemResourceName,ProductionType,InstalledGenCapacity
1186,S.M.GAROÑA,Nuclear,455.2
1353,TRILLO,Nuclear,1003.4
1588,VANDELLOS,Nuclear,1045.3


### Update the model

Add 'entsoe_names' field to the generators in the model and delete the temporary name:

In [59]:
for gen in model['gen'].values():
    gen['entsoe_names'] = name_map[gen['name']] if gen['name'] in name_map.keys() else []
    del gen['name']

# Add aggregated types

List existing generator types:

In [60]:
all_gen_types = {gen['type'] for gen in model['gen'].values()}
all_gen_types

{'Biomass',
 'Coal',
 'Gas',
 'Geothermal',
 'Hydro',
 'Nuclear',
 'Oil',
 'biomass',
 'fossil_brown_lignite',
 'fossil_brown_lignite_cons',
 'fossil_coal_gas',
 'fossil_coal_gas_cons',
 'fossil_coal_hard',
 'fossil_mixed',
 'fossil_oil',
 'hydro_mixed',
 'hydro_mixed_cons',
 'hydro_pure_ps',
 'hydro_pure_storage',
 'hydro_pure_storage_cons',
 'hydro_ror',
 'nuclear',
 'nuclear_cons',
 'other_nl',
 'other_nrenew',
 'other_nrenew_cons',
 'waste_nr'}

Manually define aggregated types:

In [61]:
types_aggregation_map = {
    "Hydro"                    : "hydro",
    "hydro_pure_storage"       : "hydro_storage",
    "hydro_pure_storage_cons"  : "hydro_storage",
    "hydro_pure_ps"            : "hydro_storage",
    "hydro_mixed"              : "hydro_storage",
    "hydro_mixed_cons"         : "hydro_storage",
    "hydro_ror"                : "hydro_ror",
    "fossil_brown_lignite"     : "coal",
    "fossil_brown_lignite_cons": "coal",
    "fossil_coal_hard"         : "coal",
    "fossil_coal_gas"          : "gas",
    "fossil_oil"               : "gas",
    "fossil_mixed"             : "gas",
    "fossil_coal_gas_cons"     : "gas",
    "Coal"                     : "coal",
    "Gas"                      : "gas",
    "Oil"                      : "gas",
    "nuclear"                  : "nuclear",
    "Nuclear"                  : "nuclear",
    "nuclear_cons"             : "nuclear",
    "other_nrenew"             : "other",
    "other_nrenew_cons"        : "other",
    "other_nl"                 : "other",
    "Geothermal"               : "other",
    "biomass"                  : "other",
    "Biomass"                  : "other",
    "Waste"                    : "gas",
    "waste_nr"                 : "gas"
}

In [62]:
aggregated_types = set(types_aggregation_map.values())
aggregated_types

{'coal', 'gas', 'hydro', 'hydro_ror', 'hydro_storage', 'nuclear', 'other'}

Check that all types are mapped: 

In [63]:
assert all_gen_types <= set(types_aggregation_map.keys())

In [64]:
for gen in model['gen'].values():
    gen['aggregated_type'] = types_aggregation_map[gen['type']]

# Define expected usage

Choose 2016 as the reference year, as it matches with the model:

In [65]:
year = 2016

### Capacity

In [66]:
capacity_df = pd.read_csv('%s/%d_01_InstalledGenerationCapacityAggregated_14.1.A.csv' % (entsoe_data_dir, year + 1),
                          sep='\t', usecols=['AreaTypeCode', 'MapCode', 'ProductionType', 'AggregatedInstalledCapacity'])
capacity_df = capacity_df[(capacity_df['AreaTypeCode'] == 'CTY') & (capacity_df['MapCode'].isin(model_countries))]

In [67]:
capacity_by_country = capacity_df.groupby('MapCode') \
    .apply(lambda group: group.set_index('ProductionType')['AggregatedInstalledCapacity'].to_dict()).to_dict()

/tmp/ipykernel_38780/1427180708.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.set_index('ProductionType')['AggregatedInstalledCapacity'].to_dict()).to_dict()


In [68]:
all_capacity_types = {type for capacity_by_type in capacity_by_country.values() for type in capacity_by_type.keys()}
all_capacity_types

{'Biomass',
 'Fossil Brown coal/Lignite',
 'Fossil Coal-derived gas',
 'Fossil Gas',
 'Fossil Hard coal',
 'Fossil Oil',
 'Fossil Oil shale',
 'Fossil Peat',
 'Geothermal',
 'Hydro Pumped Storage',
 'Hydro Run-of-river and poundage',
 'Hydro Water Reservoir',
 'Marine',
 'Nuclear',
 'Other',
 'Other renewable',
 'Solar',
 'Waste',
 'Wind Offshore',
 'Wind Onshore'}

In [69]:
capacity_by_country['CH']

{'Hydro Run-of-river and poundage': 190.0,
 'Nuclear': 3373.0,
 'Hydro Water Reservoir': 4897.8,
 'Hydro Pumped Storage': 6228.0}

In [70]:
capacity_by_country['DE']

{'Fossil Brown coal/Lignite': 21262.0,
 'Biomass': 7080.25,
 'Fossil Hard coal': 27437.0,
 'Waste': 1685.0,
 'Geothermal': 40.22,
 'Other': 1421.0,
 'Hydro Water Reservoir': 1439.0,
 'Fossil Oil': 4614.0,
 'Wind Offshore': 4131.3,
 'Fossil Coal-derived gas': 0.0,
 'Fossil Gas': 32627.0,
 'Hydro Run-of-river and poundage': 4007.13,
 'Other renewable': 509.61,
 'Hydro Pumped Storage': 8894.24,
 'Nuclear': 10793.0,
 'Wind Onshore': 47042.0,
 'Solar': 40834.46}

### Generation

In [71]:
gen_df = pd.concat([pd.read_csv('%s/%d_%02d_AggregatedGenerationPerType_16.1.B_C.csv' % (entsoe_data_dir, year, month + 1),
                                sep='\t', usecols=['AreaTypeCode', 'MapCode', 'ProductionType', 'ActualGenerationOutput'])
                    for month in range(12)], ignore_index=True)
gen_df = gen_df[(gen_df['AreaTypeCode'] == 'CTY') & (gen_df['MapCode'].isin(model_countries))]
gen_df = gen_df.groupby(['MapCode', 'ProductionType']).filter(lambda g: g['ActualGenerationOutput'].count() > 1000)

In [72]:
gen_by_country = gen_df.groupby('MapCode') \
    .apply(lambda group: group.groupby('ProductionType')['ActualGenerationOutput'].mean().to_dict()).to_dict()

/tmp/ipykernel_38780/3656648501.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.groupby('ProductionType')['ActualGenerationOutput'].mean().to_dict()).to_dict()


In [73]:
all_gen_types = {type for gen_by_type in gen_by_country.values() for type in gen_by_type.keys()}
all_gen_types

{'Biomass',
 'Fossil Brown coal/Lignite',
 'Fossil Coal-derived gas',
 'Fossil Gas',
 'Fossil Hard coal',
 'Fossil Oil',
 'Fossil Oil shale',
 'Fossil Peat',
 'Geothermal',
 'Hydro Pumped Storage',
 'Hydro Run-of-river and poundage',
 'Hydro Water Reservoir',
 'Marine',
 'Nuclear',
 'Other',
 'Other renewable',
 'Solar',
 'Waste',
 'Wind Offshore',
 'Wind Onshore'}

Check that this is the same types as for the capacity computation:

In [74]:
assert all_gen_types == all_capacity_types

In [75]:
gen_by_country['CH']

{'Hydro Pumped Storage': 584.12845856102,
 'Hydro Run-of-river and poundage': 68.68743913857678,
 'Hydro Water Reservoir': 904.7273576958106,
 'Nuclear': 2244.7649134790527,
 'Solar': 38.301363013698634,
 'Wind Onshore': 7.819798534798534}

In [76]:
gen_by_country['DE']

{'Biomass': 4524.766928790984,
 'Fossil Brown coal/Lignite': 14842.226127903006,
 'Fossil Coal-derived gas': 431.8244763205829,
 'Fossil Gas': 2605.7666140141164,
 'Fossil Hard coal': 9208.768320241348,
 'Fossil Oil': 198.01513974271404,
 'Geothermal': 19.224431067850638,
 'Hydro Pumped Storage': 977.6436233492715,
 'Hydro Run-of-river and poundage': 1964.5508122723134,
 'Hydro Water Reservoir': 93.27279684653917,
 'Nuclear': 9134.58341302368,
 'Other': 309.27984090391624,
 'Other renewable': 140.52764571948998,
 'Solar': 3932.0249339708566,
 'Waste': 625.344625170765,
 'Wind Offshore': 1376.8343317395263,
 'Wind Onshore': 7431.724159836066}

**Remark**

A few computed productions are above capacity for a given type. The problem will be solved by using aggregated types.

In [77]:
for country, gen_by_type in gen_by_country.items():
    if country in capacity_by_country:
        for type, value in gen_by_type.items():
            if type in capacity_by_country[country]:
                if value > capacity_by_country[country][type]:
                    print('Production (%.2f) above capacity (%.2f) for type "%s" in country %s'
                          % (value, capacity_by_country[country][type], type, country))

Production (6.93) above capacity (0.00) for type "Fossil Oil" in country CZ
Production (109.33) above capacity (0.00) for type "Other" in country CZ
Production (431.82) above capacity (0.00) for type "Fossil Coal-derived gas" in country DE
Production (380.71) above capacity (0.00) for type "Fossil Brown coal/Lignite" in country ES
Production (284.76) above capacity (170.00) for type "Biomass" in country FR
Production (0.27) above capacity (0.00) for type "Fossil Oil" in country GR
Production (10472.35) above capacity (1708.00) for type "Other" in country IT
Production (7.21) above capacity (2.00) for type "Biomass" in country LU
Production (11.55) above capacity (1.00) for type "Solar" in country LU
Production (3912.27) above capacity (1.00) for type "Other" in country NL


### Aggregation

Aggregate production types:

In [78]:
entsoe_types_aggregation_map = {
    'Fossil Brown coal/Lignite'       : 'coal',
    'Fossil Hard coal'                : 'coal',
    'Fossil Coal-derived gas'         : 'gas',
    'Fossil Gas'                      : 'gas',
    'Fossil Oil'                      : 'gas',
    'Fossil Oil shale'                : 'gas',
    'Waste'                           : 'gas',
    'Fossil Peat'                     : 'coal',
    'Hydro Pumped Storage'            : 'hydro_storage',
    'Hydro Water Reservoir'           : 'hydro_storage',
    'Hydro Run-of-river and poundage' : 'hydro_ror',
    'Nuclear'                         : 'nuclear',
    'Biomass'                         : 'other',
    'Geothermal'                      : 'other',
    'Marine'                          : 'other',
    'Other'                           : 'other',
    'Other renewable'                 : 'other',
    'Solar'                           : 'other',
    'Wind Offshore'                   : 'other',
    'Wind Onshore'                    : 'other'
}

Check that all production types are mapped:

In [79]:
assert set(entsoe_types_aggregation_map.keys()) == all_gen_types

Check that the mapped types are the same as the aggregated types defined above, with the exception of "hydro":

In [80]:
assert set(entsoe_types_aggregation_map.values()) == aggregated_types - {'hydro'}

Compute aggregated capacity by country:

In [81]:
aggregated_capacity_by_country = {}

for country, capacity_by_type in capacity_by_country.items():
    types = {entsoe_types_aggregation_map[t] for t in capacity_by_type.keys()}
    
    aggregated_capacity_by_type = {t: 0.0 for t in types}
    for type, value in capacity_by_type.items():
        aggregated_capacity_by_type[entsoe_types_aggregation_map[type]] += value

    hydro = 0.0
    if 'hydro_storage' in types:
        hydro += aggregated_capacity_by_type['hydro_storage']
    if 'hydro_ror' in types:
        hydro += aggregated_capacity_by_type['hydro_ror']
    if hydro > 0.0:
        aggregated_capacity_by_type['hydro'] = hydro

    aggregated_capacity_by_country[country] = aggregated_capacity_by_type

In [82]:
aggregated_gen_by_country = {}

for country, gen_by_type in gen_by_country.items():
    types = {entsoe_types_aggregation_map[t] for t in gen_by_type.keys()}
    
    aggregated_gen_by_type = {t: 0.0 for t in types}
    for type, value in gen_by_type.items():
        aggregated_gen_by_type[entsoe_types_aggregation_map[type]] += value

    hydro = 0.0
    if 'hydro_storage' in types:
        hydro += aggregated_gen_by_type['hydro_storage']
    if 'hydro_ror' in types:
        hydro += aggregated_gen_by_type['hydro_ror']
    if hydro > 0.0:
        aggregated_gen_by_type['hydro'] = hydro

    aggregated_gen_by_country[country] = aggregated_gen_by_type

### Capacity usage

Compute usage:

In [83]:
usage_by_country = {country: {type: aggregated_gen_by_country[country][type] / value
                              for type, value in capacity_by_type.items() if type in aggregated_gen_by_country[country]}
                    for country, capacity_by_type in aggregated_capacity_by_country.items() if country in aggregated_gen_by_country}

In [84]:
for country, usage_by_type in usage_by_country.items():
    for value in usage_by_type.values():
        assert value <= 1.0

In [85]:
usage_by_country['CH']

{'nuclear': 0.665509906160407,
 'hydro_ror': 0.36151283757145675,
 'hydro_storage': 0.13382011327336737,
 'hydro': 0.13764322941333423}

In [86]:
usage_by_country['DE']

{'hydro_ror': 0.49026380783062024,
 'gas': 0.09918694074007549,
 'coal': 0.4938703966846209,
 'other': 0.17548570983031953,
 'nuclear': 0.8463433163183248,
 'hydro_storage': 0.10363800900741786,
 'hydro': 0.21167286705071936}

### Sum across countries

In [87]:
total_aggregated_capacity = {type: 0.0 for type in aggregated_types}

for country, capacity_by_type in aggregated_capacity_by_country.items():
    for type, value in capacity_by_type.items():
        total_aggregated_capacity[type] += value

total_aggregated_capacity

{'hydro_ror': 43677.45,
 'gas': 192906.67,
 'coal': 141963.96,
 'hydro': 147369.24,
 'nuclear': 103160.18999999999,
 'other': 224067.88,
 'hydro_storage': 103691.79}

In [88]:
total_aggregated_gen = {type: 0.0 for type in aggregated_types}

for country, gen_by_type in aggregated_gen_by_country.items():
    for type, value in gen_by_type.items():
        total_aggregated_gen[type] += value

total_aggregated_gen

{'hydro_ror': 18266.386970728978,
 'gas': 36266.15376996444,
 'coal': 62853.32941117908,
 'hydro': 32229.800626388787,
 'nuclear': 76172.51808017504,
 'other': 59508.86335331817,
 'hydro_storage': 13963.413655659808}

In [89]:
total_aggregated_usage = {type: total_aggregated_gen[type] / value for type, value in total_aggregated_capacity.items()}
total_aggregated_usage

{'hydro_ror': 0.41821092968405843,
 'gas': 0.18799844385870348,
 'coal': 0.44274144938742965,
 'hydro': 0.21870100318349195,
 'nuclear': 0.7383906338305023,
 'other': 0.26558408707806835,
 'hydro_storage': 0.13466267344463634}

In [90]:
total_aggregated_usage = {type: total_aggregated_gen[type] / value for type, value in total_aggregated_capacity.items()}
total_aggregated_usage

{'hydro_ror': 0.41821092968405843,
 'gas': 0.18799844385870348,
 'coal': 0.44274144938742965,
 'hydro': 0.21870100318349195,
 'nuclear': 0.7383906338305023,
 'other': 0.26558408707806835,
 'hydro_storage': 0.13466267344463634}

### Update the model

In [91]:
for gen in model['gen'].values():
    usage = usage_by_country[gen['country']][gen['aggregated_type']] \
        if gen['country'] in usage_by_country and gen['aggregated_type'] in usage_by_country[gen['country']] \
        else total_aggregated_usage[gen['aggregated_type']]
    gen['pexp'] = gen['pmax'] * usage

# Summary table

In [92]:
{key: len(values) for key, values in model.items() if isinstance(values, dict)}

{'bus': 4097,
 'dcline': 0,
 'gen': 815,
 'branch': 8375,
 'storage': 0,
 'switch': 0,
 'shunt': 0,
 'load': 4097}

In [93]:
sorted(model_countries)

['AL',
 'AT',
 'BA',
 'BE',
 'BG',
 'CH',
 'CZ',
 'DE',
 'DK',
 'ES',
 'FR',
 'GR',
 'HR',
 'HU',
 'IT',
 'LU',
 'ME',
 'MK',
 'NL',
 'PL',
 'PT',
 'RO',
 'RS',
 'SI',
 'SK',
 'XX']

In [94]:
gen_summary_values = {country: {type: [0, 0, 0] for type in ['nuclear', 'coal', 'gas', 'hydro_storage', 'hydro_ror', 'hydro', 'other']}
                      for country in sorted(model_countries)}

for gen in model['gen'].values():
    country = gen['country']
    type = gen['aggregated_type']
    gen_summary_values[country][type][0] += 1
    gen_summary_values[country][type][1] += 100 * gen['pexp']
    gen_summary_values[country][type][2] += 100 * gen['pmax']

gen_summary_total = {type: [0, 0, 0] for type in ['nuclear', 'coal', 'gas', 'hydro_storage', 'hydro_ror', 'hydro', 'other']}

for country, gen_summary_by_country in gen_summary_values.items():
    for type, gen_summary_by_type in gen_summary_by_country.items():
        gen_summary_total[type][0] += gen_summary_by_type[0]
        gen_summary_total[type][1] += gen_summary_by_type[1]
        gen_summary_total[type][2] += gen_summary_by_type[2]

gen_summary_values['total'] = gen_summary_total


gen_summary = {country: {type: "" for type in ['nuclear', 'coal', 'gas', 'hydro', 'hydro_storage', 'hydro_ror', 'other', 'total']}
               for country in sorted(model_countries) + ['total']}

for country, gen_summary_by_country in gen_summary_values.items():
    for type, gen_summary_by_type in gen_summary_by_country.items():
        if gen_summary_by_type[0] > 0:
            text = "%d \times %d / %d" % \
                (gen_summary_by_type[0],
                int(round(gen_summary_by_type[1] / gen_summary_by_type[0])),
                int(round(gen_summary_by_type[2] / gen_summary_by_type[0])))
            gen_summary[country][type] = "$\mathbf{%s}$" % text if country == 'total' else "$%s$" % text
    n = sum(gen_summary_by_type[0] for gen_summary_by_type in gen_summary_by_country.values())
    pexp = sum(gen_summary_by_type[1] for gen_summary_by_type in gen_summary_by_country.values())
    pmax = sum(gen_summary_by_type[2] for gen_summary_by_type in gen_summary_by_country.values())
    gen_summary[country]['total'] = "$\mathbf{%d \times %d / %d}$" % (n, int(round(pexp / n)), int(round(pmax / n)))

gen_summary_df = pd.DataFrame(gen_summary).T
gen_summary_df

,nuclear,coal,gas,hydro,hydro_storage,hydro_ror,other,total
AL,,,$1 \times 19 / 100$,,$4 \times 54 / 400$,$1 \times 209 / 500$,,$\mathbf{6 \times 74 / 367}$
AT,,$4 \times 114 / 410$,$11 \times 106 / 509$,,$15 \times 50 / 292$,$37 \times 91 / 158$,$1 \times 24 / 100$,$\mathbf{68 \times 85 / 258}$
BA,,$4 \times 167 / 376$,,,,$9 \times 75 / 180$,,$\mathbf{13 \times 103 / 241}$
BE,$2 \times 2359 / 2972$,$1 \times 246 / 556$,$11 \times 113 / 305$,,$1 \times 202 / 1164$,,$1 \times 47 / 170$,$\mathbf{16 \times 403 / 700}$
BG,$1 \times 1796 / 2000$,$2 \times 520 / 945$,$4 \times 151 / 802$,$1 \times 18 / 114$,$2 \times 52 / 268$,$4 \times 55 / 130$,,$\mathbf{14 \times 270 / 591}$
CH,$4 \times 555 / 833$,,,,$30 \times 36 / 272$,$2 \times 37 / 102$,,$\mathbf{36 \times 94 / 325}$
CZ,$2 \times 1203 / 1875$,$13 \times 239 / 501$,$1 \times 138 / 370$,,$2 \times 79 / 550$,$2 \times 69 / 246$,,$\mathbf{20 \times 297 / 611}$
DE,$9 \times 1135 / 1341$,$46 \times 532 / 1077$,$26 \times 48 / 481$,$1 \times 11 / 50$,$17 \times 50 / 481$,,$2 \times 73 / 416$,$\mathbf{101 \times 365 / 823}$
DK,,$3 \times 86 / 382$,$7 \times 70 / 455$,,,,$1 \times 23 / 88$,$\mathbf{11 \times 70 / 402}$
ES,$2 \times 1367 / 1623$,$3 \times 271 / 641$,$3 \times 150 / 845$,$29 \times 33 / 195$,,$14 \times 305 / 345$,$10 \times 170 / 707$,$\mathbf{61 \times 179 / 414}$


In [95]:
# gen_summary_df.index = ["\textbf{%s}" % country if country != 'XX' else 'TR' for country in gen_summary_df.index]
# gen_summary_df.to_latex()

# Export improved model

In [96]:
model['name'] = 'europe'

Export:

In [ ]:
model_json = json.dumps(model)
with open('../europe.json', 'w') as f:
    f.write(model_json)